In [1]:
import itertools
import json
import os
import time
import humanfriendly as humanfriendly
import numpy as np
# import pandas as pd
import workerpool as workerpool
from tqdm import tqdm
# from PIL import Image, ImageDraw
import glob
# import matplotlib.pyplot as plt
# import time
# import tensorflow_hub as hub
import tensorflow as tf
import cv2

from operator import itemgetter
from collections import OrderedDict

tf.compat.v1.disable_eager_execution()

tf.__version__
import gc
import pandas as pd
import numpy as np
import megadetector_V3.tf_detector_classes as mdc
gc.enable()
cwd = os.getcwd()
dataset = ["../Dataset/test", "../Dataset/camtrap"]
output_dir = os.path.join(cwd, "../drawn_boxes/")




/home/job/project_aai/python/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/job/project_aai/python/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/job/project_aai/python/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/job/project_aai/python/venv/lib/python3.7/s

TensorFlow version: 1.13.1
Is GPU available? tf.test.is_gpu_available: False


In [2]:
__slots__ = ['files', 'file', 'img', 'module', 'sess']

def draw_boxes(path, filename, im, result):
    print(filename)

    im_height, im_width, channels = im.shape
    detection_count = 0

    cwd = os.getcwd()
    output_dir = os.path.join(cwd, "../drawn_boxes/")

    os.chdir(output_dir)
    for index, box in enumerate(result['detection_boxes'][:5]):
        if result['detection_classes'][index] == 1 and result['detection_scores'][index] >= 0.95:
            detection_count += 1

            (left, right) = tuple(im_width * np.array(
                (box[1], box[0])
                if box[0] > box[1]
                else (box[0], box[1])
            ))

            (top, bottom) = tuple(im_height * np.array(
                (box[3], box[2])
                if box[2] > box[3]
                else (box[2], box[3])
            ))
            # if int(left) is int(right) or int(top) is int(bottom):
            cv2.rectangle(im, (int(left), int(top)), (int(right), int(bottom)), (0, 0, 255), 2)

            print(path, "\n", (im.shape), "\n", (left, right, top, bottom))

            print(im[int(top):int(bottom), int(left):int(right), :].shape)
            cv2.imwrite("".join([filename.split(".")[0], "_cropped", str(detection_count), ".jpg"]),
                        im[int(top):int(bottom), int(left):int(right)])

        if detection_count > 0:
            cv2.imwrite("".join([filename.split(".")[0], ".jpg"]), im)

        '''Trying to clear out some memory'''
        left = right = top = bottom = cropped = None
        del (left, right, top, bottom, cropped)

    detection_count = im_width = im_height = draw = im = None
    del (detection_count, im_width, im_height, draw, path, filename, im, result)

    os.chdir(cwd)

In [3]:
def fetch_filenames(dir):
    image_file_names = mdc.ImagePathUtils.find_images(dir)
    return image_file_names

filenames = fetch_filenames(dataset[1])
# print(filenames[0])
# print(len(filenames))
# cv2.imshow("image 0", cv2.imread(filenames[0]))
# cv2.waitKey(0)
# cv2.destroyAllWindows()


def resize_img(image,size=(512, 512)):
    height, width, channels = image.shape
    # print(height, width)

    if(height > width):
        ratio = width/height
        # print(ratio)
        dim = (int(size[0]*ratio), size[1])

    else:
        ratio = height/width
        # print(ratio)
        dim = (size[0], int(size[1]*ratio))


    image = cv2.resize(image, dim)
    # create new image of desired size and color (black) for padding
    new_heigth = size[1]
    new_width = size[0]
    color = (0,0,0)
    result = np.full((new_heigth,new_width,channels), color, dtype=np.uint8)

    # compute center offset
    xx = (new_width - dim[0]) // 2
    yy = (new_heigth - dim[1]) // 2

    # copy img image into center of result image
    result[yy:yy+dim[1], xx:xx+dim[0]] = image
    return result


def read_file(filename):
    image = cv2.imread(filename)

    image = resize_img(image)
    return image

img = read_file(filenames[-15])
cv2.imshow("image 0: {}".format((img.shape)), img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
from functools import partial


def process_images(im_files, tf_detector, confidence_threshold):

    if isinstance(tf_detector,str):
        start_time = time.time()
        tf_detector = mdc.TFDetector(tf_detector)
        elapsed = time.time() - start_time
        print('Loaded model (batch level) in {}'.format(humanfriendly.format_timespan(elapsed)))

    results = []
    for im_file in im_files:
        results.append(process_image(im_file, tf_detector, confidence_threshold))
    return results


def process_image(im_file, tf_detector, confidence_threshold):

    if isinstance(tf_detector,str):
        start_time = time.time()
        tf_detector = mdc.TFDetector(tf_detector)
        elapsed = time.time() - start_time
        print('Loaded model (worker level) in {}'.format(humanfriendly.format_timespan(elapsed)))

    print('Processing image {}'.format(im_file))
    image = None
    try:
        image = read_file(im_file)
    except Exception as e:
        print('Image {} cannot be loaded. Exception: {}'.format(im_file, e))
        result = {
            'file': im_file,
            'failure': mdc.TFDetector.FAILURE_IMAGE_OPEN
        }
        return result

    try:
        result = tf_detector.generate_detections_one_image(image, im_file,
                                                           detection_threshold=confidence_threshold)
    except Exception as e:
        print('Image {} cannot be processed. Exception: {}'.format(im_file, e))
        result = {
            'file': im_file,
            'failure': mdc.TFDetector.FAILURE_TF_INFER
        }
        return result

    return result


# Split a list into chunks of size n
def chunks_by_size(l, n):
    return [l[i:i+n] for i in range(0, len(l), n)]

# Split a list into n even chunks
def chunks_by_number_of_chunks(l, n):
    for i in range(0, n):
        yield l[i::n]



def load_and_run_detector_batch(model_file, image_file_names, checkpoint_path=None,
                                confidence_threshold=0, checkpoint_frequency=-1,
                                results=None, n_cores=0):

    if results is None:
        results = []
    #
    # already_processed = set([i['file'] for i in results])

    if n_cores > 1 and tf.test.is_gpu_available():
        print('Warning: multiple cores requested, but a GPU is available; parallelization across GPUs is not currently supported, defaulting to one GPU')

    # If we're not using multiprocessing...
    if n_cores <= 1 or tf.test.is_gpu_available():
        # Load the detector
        start_time = time.time()
        tf_detector = mdc.TFDetector(model_file)
        elapsed = time.time() - start_time
        print('Loaded model in {}'.format(humanfriendly.format_timespan(elapsed)))
    else:
        # If we're using multiprocessing, let the workers load the model, just store
        # the model filename.
        tf_detector = model_file

    if n_cores <= 1 or tf.test.is_gpu_available():

        # Does not count those already processed
        count = 0

        for im_file in tqdm(image_file_names):

            # # Will not add additional entries not in the starter checkpoint
            # if im_file in already_processed:
            #     print('Bypassing image {}'.format(im_file))
            #     continue

            count += 1

            try:
                image = read_file(im_file)
            except Exception as e:
                print('Image {} cannot be loaded. Exception: {}'.format(im_file, e))
                result = {
                    'file': im_file,
                    'failure': mdc.TFDetector.FAILURE_IMAGE_OPEN
                }
                results.append(result)
                continue

            try:
                result = tf_detector.generate_detections_one_image(image, im_file, detection_threshold=confidence_threshold)
                results.append(result)

            except Exception as e:
                print('An error occurred while running the detector on image {}. Exception: {}'.format(im_file, e))
                result = {
                    'file': im_file,
                    'failure': mdc.TFDetector.FAILURE_TF_INFER
                }
                results.append(result)
                continue

            # checkpoint
            if checkpoint_frequency != -1 and count % checkpoint_frequency == 0:
                print('Writing a new checkpoint after having processed {} images since last restart'.format(count))
                with open(checkpoint_path, 'w') as f:
                    json.dump({'images': results}, f)
    else:
        print('Creating pool with {} cores'.format(n_cores))
        #
        # if len(already_processed) > 0:
        #     print('Warning: when using multiprocessing, all images are reprocessed')

        pool = workerpool.WorkerPool(n_cores)

        image_batches = list(chunks_by_number_of_chunks(image_file_names,n_cores))
        results = pool.map(partial(process_images, tf_detector=tf_detector,
                                    confidence_threshold=confidence_threshold), image_batches)

        results = list(itertools.chain.from_iterable(results))

    # This was modified in place, but we also return it for backwards-compatibility.
    return results


In [ ]:

# model_file = "../models/megadetector_V3/saved_model/../models/megadetector_V3/saved_model/saved_model_normalized_megadetector_v3_tf19/"
model_file = "../models/megadetector_V3/saved_model/megadetector_v3.pb"
res = load_and_run_detector_batch(model_file, filenames[:100], n_cores=0, confidence_threshold=0.85)





TFDetector: Loading graph...
TFDetector: Detection graph loaded.
Loaded model in 6.42 seconds


  2%|▏         | 2/100 [00:17<16:55, 10.36s/it]

In [ ]:
res_df = pd.DataFrame(res)
res

In [ ]:
with open("./megadetector_results_85.json", "w") as outfile:
    json.dump(res, outfile)

In [5]:

with open("./megadetector_results_85.json", "r") as json_results:
    data = json.load(json_results)
res = data

In [6]:
def draw_box(file_path, bbox, cat, conf, img):
    if img is None:
        image = read_file(file_path)
    else:
        image = img
    # cv2.imshow("file: {}".format("ddddd"), image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    im_height, im_width, channels = image.shape
    """[x1, y1, width_box, height_box]"""
    (x1, y1, width_box, height_box) = bbox
    (left, right, top, bottom) = (x1, x1+width_box, y1, y1+height_box)
    # (left, right, top, bottom) = bbox
    (left, right) = im_width*np.array((left, right))
    (top, bottom) = im_height*np.array((top, bottom))
    print((left, right), (top, bottom))
    # (left, right) = tuple(im_width * np.array(
    #     (bbox[1], bbox[0])
    #     if bbox[0] > bbox[1]
    #     else (bbox[0], bbox[1])
    # ))

    # (top, bottom) = tuple(im_height * np.array(
    #     (bbox[3], bbox[2])
    #     if bbox[2] > bbox[3]
    #     else (bbox[2], bbox[3])
    # ))
    image = cv2.rectangle(image, (int(left), int(top)), (int(right), int(bottom)), (0, 0, 255), 2)
    # cv2.putText(image, 'class: {}, Confidence:{}'.format(cat, conf), (int(left), int(top-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.50, (0, 0, 255), 1)
    return image

In [7]:
def crop_animal(original_img, bbox):
    im_height, im_width, channels = original_img.shape
    """[x1, y1, width_box, height_box]"""
    (x1, y1, width_box, height_box) = bbox
    (left, right, top, bottom) = (x1, x1+width_box, y1, y1+height_box)
    # (left, right, top, bottom) = bbox
    (left, right) = im_width*np.array((left, right))
    (top, bottom) = im_height*np.array((top, bottom))
    print((left, right), (top, bottom))
    return original_img[int(top):int(bottom), int(left):int(right)]

In [8]:

for i in tqdm(res):
    if len(i['detections']) > 0:
        print(i['file'])
        path_to_save = "../megadetector_result/{}".format(i['file'].split("/")[-1].split(".")[0])
        # create a dir for this particular image to hold all the processed images
        # os.mkdir(path_to_save)
        try:
            os.mkdir(path_to_save)
        except OSError as error:
            print(error)
        try:
            os.mkdir(path_to_save+"/cropped")
        except OSError as error:
            print(error)
        # save the output json of that image in this folder
        with open(path_to_save+"/raw_result.json", "w") as outfile:
            json.dump(i, outfile)
        returned_image = None
        original_image = cv2.imread(i['file'])

        for index, det in enumerate(i['detections']):
            if det['category'] is '1':
                returned_image = draw_box(i['file'],det['bbox'], det['category'], det['conf'], returned_image)
                cropped = crop_animal(original_image, det['bbox'])
                # save the returned cropped image
                cv2.imwrite(path_to_save+"/cropped/cropped_{}.jpg".format(index), cropped)
                # cv2.imshow("cropped_{}".format(index), cropped)
                # cv2.waitKey(0)
                # cv2.destroyAllWindows()
                print(det)
        # save the original image with bounding boxes
        if returned_image is not None:        # save the original image
            cv2.imwrite(path_to_save+"/original.jpg", original_image)
            cv2.imwrite(path_to_save+"/original_bboxes.jpg", returned_image)


100%|██████████| 963/963 [11:02<00:00,  1.45it/s]


../Dataset/camtrap/11180312.JPG
[Errno 17] File exists: '../megadetector_result/11180312'
[Errno 17] File exists: '../megadetector_result/11180312/cropped'
(191.5392, 385.0752) (125.952, 412.16)
(718.2719999999999, 1444.032) (354.24, 1159.2)
{'category': '1', 'conf': 0.999, 'bbox': [0.3741, 0.246, 0.378, 0.559]}
../Dataset/camtrap/07020365.JPG
[Errno 17] File exists: '../megadetector_result/07020365'
[Errno 17] File exists: '../megadetector_result/07020365/cropped'
(215.808, 402.99519999999995) (169.1648, 350.72)
(809.28, 1511.2319999999997) (475.77600000000007, 986.4000000000001)
{'category': '1', 'conf': 0.999, 'bbox': [0.4215, 0.3304, 0.3656, 0.3546]}
../Dataset/camtrap/EK000359.JPG
[Errno 17] File exists: '../megadetector_result/EK000359'
[Errno 17] File exists: '../megadetector_result/EK000359/cropped'
(376.1664, 510.66880000000003) (154.88, 417.6384)
(1504.6656, 2042.6752000000001) (464.64, 1252.9152)
{'category': '1', 'conf': 0.999, 'bbox': [0.7347, 0.3025, 0.2627, 0.5132]}
../D